In [1]:
import pandas as pd


In [2]:
df1 = pd.read_csv('/Users/bhanukedhar/Documents/Visual Analytics/Project-1/my-visual-analytics-app/public/simplemaps_uscities_basicv1/uscities.csv')

In [3]:
df2 = pd.read_csv("/Users/bhanukedhar/Documents/Visual Analytics/Project-1/my-visual-analytics-app/public/bank_transactions_data_2.csv")

In [4]:
df1

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18832416,10943.7,shape,False,True,America/New_York,1,11229 11226 11225 11224 11223 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,11885717,3165.8,shape,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90035 90034 9003...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8489066,4590.3,shape,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6113982,4791.1,shape,False,True,America/New_York,1,33128 33129 33125 33126 33127 33149 33144 3314...,1840015149
4,Houston,Houston,TX,Texas,48201,Harris,29.7860,-95.3885,6046392,1386.5,shape,False,True,America/Chicago,1,77069 77068 77061 77060 77063 77062 77065 7706...,1840020925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31249,Mount Wilson,Mount Wilson,NV,Nevada,32017,Lincoln,38.2350,-114.4504,0,2.2,shape,False,True,America/Los_Angeles,5,89043,1840039843
31250,Kohatk,Kohatk,AZ,Arizona,4021,Pinal,32.5781,-112.0032,0,77.3,shape,False,False,America/Phoenix,3,85634,1840022983
31251,Ironville,Ironville,PA,Pennsylvania,42013,Blair,40.6586,-78.2155,0,59.2,shape,False,False,America/New_York,3,16686,1840152922
31252,Newkirk,Newkirk,NM,New Mexico,35019,Guadalupe,35.0635,-104.2715,0,0.0,shape,False,False,America/Denver,3,88417,1840024978


In [5]:

# Create helper columns for case-insensitive matching
df1["city_lower"] = df1["city"].str.lower()
df1["city_ascii_lower"] = df1["city_ascii"].str.lower()
df2["Location_lower"] = df2["Location"].str.lower()

# Convert population column to numeric (in case it's a string)
df1["population"] = pd.to_numeric(df1["population"], errors='coerce')

# For each unique city (using city_lower), keep only the entry with the highest population.
# This assumes that the major city is the one you're interested in.
df1_unique = df1.sort_values("population", ascending=False).drop_duplicates(subset=["city_lower"])

# First, merge on 'city_lower'
merged = pd.merge(
    df2,
    df1_unique[["city_lower", "state_id", "lat", "lng"]],
    left_on="Location_lower",
    right_on="city_lower",
    how="left"
)

# For rows that didn't match, try merging on 'city_ascii_lower'
missing = merged[merged["lat"].isna()].copy()
if not missing.empty:
    missing = missing.drop(columns=["lat", "lng"])
    merged_missing = pd.merge(
        missing,
        df1_unique[["city_ascii_lower", "state_id", "lat", "lng"]],
        left_on="Location_lower",
        right_on="city_ascii_lower",
        how="left"
    )
    merged.update(merged_missing)


In [7]:
merged.to_csv("/Users/bhanukedhar/Documents/Visual Analytics/Project-1/my-visual-analytics-app/public/bank_transactions_data_2.csv")